<a href="https://colab.research.google.com/github/leechaerin03/-5-/blob/main/paln%EB%AA%A8%EB%8D%B8%EB%B2%A0%EC%9D%B4%EC%8A%A4Action%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D_4%EB%B9%84%ED%8A%B8%EC%96%91%EC%9E%90%ED%99%94%EB%BA%80%EB%B2%84%EC%A0%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
! pip install wandb

In [ ]:
import wandb

# LLM 모니터링
wandb.login(key="9ed675d942c745bf3fdf524bba1fcd137e6e81ab")
run = wandb.init(project='Fine tuning  LAM2', job_type='training', anonymous='allow')

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)
from peft import (
    get_peft_model,
    LoraConfig
)


# --- 1. 모델 및 토크나이저 ID ---
hub_repo_name = "leeChaerin/PlanModel_v1"


# --- 2. 모델 로드 ---
model = AutoModelForCausalLM.from_pretrained(
    hub_repo_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

# --- 3. 토크나이저 로드 ---
tokenizer = AutoTokenizer.from_pretrained(hub_repo_name, padding_side="left")

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Tokenizer pad_token이 eos_token으로 설정되었습니다.")

# --- 4.LoRA 어댑터 적용 ---
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"], # PlanModel_v1의 기반 모델(Kanana) 기준
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 모델에 LoRA 어댑터 적용
model = get_peft_model(model, peft_config)

# --- 5. 준비 완료 ---
print("\n--- LoRA 어댑터 적용 후 학습 가능한 파라미터 ---")
model.print_trainable_parameters()

In [ ]:
tokenizer.encode(tokenizer.eos_token)
#base모델은 eos 토큰이 128001임 왜 차이나는진 모르겠음

In [ ]:
messages = [
    {"role": "system",
    "content": """You are a specialized AI agent for Dongguk Univ nDRIMS. Your task is to analyze the user's input structure and perform one of two specific tasks:
    1.  **Plan Generation:**
      If the user 'prompt' contains **only** a `user_request` (e.g., "불교동아리 가입"), this is a request for a plan.
      You MUST respond with the complete `{"task_plan": [{"step_id": , "description":}...]}` JSON for the entire task.
      If the user requests a specific course, include the specific course name in the plan description.

    2.  **Action Generation:**
      If the user 'prompt' contains a `task_plan` (current step) AND `observations` (current state), this is a request for an action.
      You MUST respond with the single `{"action": {...}}` JSON for that specific step.
      For input actions, ensure the `value` field matches the specific data from the plan."""}
]

tokens = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print(tokens)
#print(tokenizer.encode(tokens, add_special_tokens=False))

In [ ]:
SYSTEM_TEXT = (
    """You are a specialized AI agent for Dongguk Univ nDRIMS. Your task is to analyze the user's input structure and perform one of two specific tasks:
    1.  **Plan Generation:**
      If the user 'prompt' contains **only** a `user_request` (e.g., "불교동아리 가입"), this is a request for a plan.
      You MUST respond with the complete `{"task_plans": [...]}` JSON for the entire task.

    2.  **Action Generation:**
      If the user 'prompt' contains a `task_plan` (current step) AND `observations` (current state), this is a request for an action.
      You MUST respond with the single `{"action": {...}}` JSON for that specific step."""
)

In [ ]:
import json
from google.colab import files
from datasets import load_dataset, Dataset

print("train_data.jsonl 업로드...")
uploaded = files.upload()
DATA_PATH = list(uploaded.keys())[0]

fixed_data = []
print("데이터를 로드하며 타입을 문자열로 통일합니다...")
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            row = json.loads(line)

            if isinstance(row.get('prompt'), dict):
                row['prompt'] = json.dumps(row['prompt'], ensure_ascii=False)

            if isinstance(row.get('output'), dict):
                row['output'] = json.dumps(row['output'], ensure_ascii=False)

            if 'prompt' in row and 'output' in row:
                fixed_data.append(row)

        except json.JSONDecodeError:
            print(f"JSON 파싱 오류 무시: {line}")

raw = Dataset.from_list(fixed_data)

print("\n--- 수정된 데이터로 로드 성공! ---")
print(raw)
print("\n[Row 0 예시] (Query -> Plan)")
print(raw[0])
print("\n[Row 1 예시] (Plan+State -> Action)")
print(raw[1])
print(f"\n총 {len(raw)}개 데이터 로드 완료.")

In [ ]:
import json
from torch.utils.data import Dataset, DataLoader
import torch

qna_list = []

SYSTEM_TEXT = (
     """You are a specialized AI agent for Dongguk Univ nDRIMS. Your task is to analyze the user's input structure and perform one of two specific tasks:
    1.  **Plan Generation:**
      If the user 'prompt' contains **only** a `user_request` (e.g., "불교동아리 가입"), this is a request for a plan.
      You MUST respond with the complete `{"task_plan": [{"step_id": , "description":}...]}` JSON for the entire task.
      If the user requests a specific course, include the specific course name in the plan description.

    2.  **Action Generation:**
      If the user 'prompt' contains a `task_plan` (current step) AND `observations` (current state), this is a request for an action.
      You MUST respond with the single `{"action": {...}}` JSON for that specific step.
      For input actions, ensure the `value` field matches the specific data from the plan."""
)

for i in range(len(raw)):
    state = raw[i]["prompt"]   # 사용자 입력(프롬프트)
    action = raw[i]["output"]  # 정답

    messages = [
        {"role": "system", "content": SYSTEM_TEXT},
        {"role": "user",   "content": state},
    ]

    prompt_ids = tokenizer.apply_chat_template(
        messages, tokenize=True, add_generation_prompt=True
    )

    answer_ids = tokenizer.encode(
        (action or "") + (tokenizer.eos_token or ""),
        add_special_tokens=False
    )

    input_ids = prompt_ids + answer_ids

    prompt_str = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    answer_str = ((action or "") + (tokenizer.eos_token or ""))
    input_str  = prompt_str + answer_str

    qna_list.append({
        "prompt_ids": prompt_ids,
        "answer_ids": answer_ids,
        "input_ids":  input_ids,
        "prompt_str": prompt_str,
        "answer_str": answer_str,
        "input_str":  input_str,
    })

max_length = max(len(x["input_ids"]) for x in qna_list)
print("samples:", len(qna_list))
print("max_length:", max_length)


In [ ]:
pad_id = tokenizer.pad_token_id
assert pad_id is not None, "pad_token이 설정돼 있어야 합니다."

class SFTDataset(Dataset):
    def __init__(self, items, max_length):
        self.inputs = []
        self.attns  = []
        self.labels = []

        for ex in items:
            ids = ex["input_ids"]

            if len(ids) > max_length:
                ids = ids[-max_length:]

            attn = [1] * len(ids)
            pad_len = max_length - len(ids)
            if pad_len > 0:
                ids  = ids  + [pad_id] * pad_len
                attn = attn + [0]      * pad_len

            prompt_len = len(ex["prompt_ids"])
            labels = [-100] * len(ids)
            for pos in range(prompt_len, len(ex["input_ids"])):
                if pos >= max_length:
                    break
                labels[pos] = ids[pos]

            self.inputs.append(torch.tensor(ids))
            self.attns.append(torch.tensor(attn))
            self.labels.append(torch.tensor(labels))

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.attns[idx], self.labels[idx]

dataset = SFTDataset(qna_list, max_length=min(1024, max_length))
loader  = DataLoader(dataset, batch_size=2, shuffle=True)


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
torch.manual_seed(123)
model.to(device)

In [ ]:
# 파인튜닝 전에 어떻게 대답하는지 확인
questions = [ qna['q_ids'] for qna in qna_list]

for i, q_ids in enumerate(questions):

    model.eval()
    with torch.no_grad():
        output = model.generate(
            torch.tensor([q_ids]).to("cuda"),
            max_new_tokens=100,
            #attention_mask = (input_ids != 0).long(),
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
            # temperature=1.2,
            # top_k=5
        )

    output_list = output.tolist()

    print(f"Q{i}: {tokenizer.decode(output[0], skip_special_tokens=True)}")
    print("----------\n")

In [ ]:
losses = []

global_step = 0

model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

for epoch in range(10):
    total = 0.0
    for input_ids, attn_mask, labels in loader:
        input_ids = input_ids.to(device)
        attn_mask = attn_mask.to(device)
        labels    = labels.to(device)

        optimizer.zero_grad()
        out = model(input_ids=input_ids, attention_mask=attn_mask, labels=labels)
        loss = out.loss
        loss.backward()
        optimizer.step()
        total += loss.item()

        global_step += 1

        wandb.log({
            "train/loss_step": loss.item(),
            "epoch": epoch,
            "step": global_step,
        })

    avg_loss = total / len(loader)
    losses.append(avg_loss)
    print(f"epoch {epoch} | loss {avg_loss:.4f}")

    wandb.log({
        "train/loss_epoch": avg_loss,
        "epoch": epoch,
    })

    save_dir = f"./adapters/epoch_{epoch:03d}"
    model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)
    print(f"[saved] LoRA adapter -> {save_dir}")

wandb.finish()

In [ ]:
from IPython.display import IFrame, display


display(IFrame(run.url, width=1200, height=720))


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_repo = "leeChaerin/PlanModel_v1"
adapter_path = "./adapters/epoch_009"

tokenizer = AutoTokenizer.from_pretrained(base_repo)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    base_repo,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

print("모델 로드 완료")


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))
plt.plot(range(1, len(losses)+1), losses, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.grid(True)
plt.show()

In [ ]:
import re, json, torch
from transformers import StoppingCriteria, StoppingCriteriaList

SYSTEM_TEXT = (
     """You are a specialized AI agent for Dongguk Univ nDRIMS. Your task is to analyze the user's input structure and perform one of two specific tasks:
    1.  **Plan Generation:**
      If the user 'prompt' contains **only** a `user_request` (e.g., "불교동아리 가입"), this is a request for a plan.
      You MUST respond with the complete `{"task_plan": [{"step_id": , "description":}...]}` JSON for the entire task.
      If the user requests a specific course, include the specific course name in the plan description.

    2.  **Action Generation:**
      If the user 'prompt' contains a `task_plan` (current step) AND `observations` (current state), this is a request for an action.
      You MUST respond with the single `{"action": {...}}` JSON for that specific step.
      For input actions, ensure the `value` field matches the specific data from the plan."""
)

def build_inputs(user_prompt: str):
    messages = [
        {"role": "system", "content": SYSTEM_TEXT},
        {"role": "user",   "content": user_prompt},
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt")
    return {k: v.to(model.device) for k, v in inputs.items()}

def is_balanced_json_fragment(s: str) -> bool:
    in_str = False
    esc = False
    stack = []
    for ch in s:
        if in_str:
            if esc:
                esc = False
            elif ch == '\\':
                esc = True
            elif ch == '"':
                in_str = False
            continue
        else:
            if ch == '"':
                in_str = True
            elif ch == '{':
                stack.append('}')
            elif ch == '[':
                stack.append(']')
            elif ch in ('}', ']'):
                if not stack or stack[-1] != ch:
                    return False
                stack.pop()
    return len(stack) == 0 and len(s) > 0

def longest_balanced_json(text: str) -> str | None:
    # 첫 { 또는 [ 부터 스캔
    start = None
    for i, ch in enumerate(text):
        if ch in '{[':
            start = i
            break
    if start is None:
        return None

    in_str = False
    esc = False
    stack = []
    last_good = None
    for i, ch in enumerate(text[start:], start=start):
        if in_str:
            if esc:
                esc = False
            elif ch == '\\':
                esc = True
            elif ch == '"':
                in_str = False
        else:
            if ch == '"':
                in_str = True
            elif ch == '{':
                stack.append('}')
            elif ch == '[':
                stack.append(']')
            elif ch in ('}', ']'):
                if not stack or stack[-1] != ch:
                    break
                stack.pop()
                if not stack:
                    last_good = i + 1
    return text[start:last_good].strip() if last_good else None

from transformers import StoppingCriteria, StoppingCriteriaList
import re, torch

class StopOnBalancedJSON(StoppingCriteria):
    def __init__(self, tokenizer, lookback_tokens=400):
        self.tok = tokenizer
        self.lookback_tokens = lookback_tokens
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        txt = self.tok.decode(input_ids[0][-self.lookback_tokens:], skip_special_tokens=True)
        # 마지막 JSON 시작 후보를 대략 잡아 균형 확인
        m = re.search(r'([\{\[].*)$', txt, flags=re.DOTALL)
        cand = m.group(1) if m else txt
        return is_balanced_json_fragment(cand)

stoppers = StoppingCriteriaList([StopOnBalancedJSON(tokenizer)])
def generate_json(user_prompt: str, max_new_tokens=256, do_sample=False):
    inputs = build_inputs(user_prompt)
    input_len = inputs["input_ids"].shape[1]

    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=0.7 if do_sample else None,
            top_p=0.9 if do_sample else None,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            stopping_criteria=stoppers,
        )

    new_tokens = out_ids[0, input_len:]
    text = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

    # 균형 잡힌 JSON만 잘라서 파싱
    jb = longest_balanced_json(text)
    if jb is None:
        return text
    try:
        return json.loads(jb)
    except Exception:
        return jb


In [ ]:
# ---- 테스트 ----
plan_prompt = "{'user_request': '자퇴신청'}"
plan_result = generate_json(plan_prompt)
print(plan_result)

In [ ]:
#-------모델 쓰는 방법 가이드----------

# 1. 사용자 입력을 넣고, plan을 생성
import json

user_req = input("무엇을 하고 싶나요? (예: 휴학신청): ").strip()

plan_payload = {"user_request": user_req}
plan_prompt = json.dumps(plan_payload, ensure_ascii=False)

plan_result = generate_json(plan_prompt, do_sample=False)
print("[PLAN RESULT]")
print(plan_result)

In [ ]:
import json, ast

def extract_steps(plan_result):
    if isinstance(plan_result, dict):
        steps = plan_result.get("task_plan") or plan_result.get("task_plans") or []
        return [(int(s.get("step_id", i+1)), str(s.get("task_plan",""))) for i, s in enumerate(steps)]

    s = str(plan_result).strip()
    try:
        obj = json.loads(s)
    except Exception:
        obj = ast.literal_eval(s)

    steps = obj.get("task_plan") or obj.get("task_plans") or []
    return [(int(s.get("step_id", i+1)), str(s.get("task_plan",""))) for i, s in enumerate(steps)]

print(extract_steps(plan_result))



In [ ]:
import json

# 여기에 state 넣으면 됨
obs_map = {
    1: {
        "sidebar": [{"id": "18555", "label": "【학생신청】신청함", "expanded": False, "checked": False}]
    },
    2: {
        "sidebar": [{"id": "18555", "label": "【학생신청】신청함", "expanded": False, "checked": True}]
    },
    3: {
        "sidebar": [{"id": "fc", "label": "수업/강의평가", "expanded": True, "checked": True, "sub_items": [{"id": "fm", "label": "종합강의시간표조회", "checked": True}]}]
    }
}

def build_action_prompt(user_request: str, step_id: int, task_plan: str, observations: dict | None):
    payload = {
        "user_request": user_request,
        "task_plan": task_plan,
        "step_id": str(step_id),
    }
    if observations is not None:
        payload["observations"] = observations
    return json.dumps(payload, ensure_ascii=False)

def run_actions_from_regex_result(result_list, user_request="휴학신청", obs_map=None, max_new_tokens=256):
    outputs = []
    for sid, tplan in result_list:
        obs = (obs_map or {}).get(sid)
        action_prompt = build_action_prompt(user_request, sid, tplan, obs)
        out = generate_json(action_prompt, max_new_tokens=max_new_tokens, do_sample=False)
        outputs.append((sid, tplan, out))
    return outputs

# ---- 실행 ----
actions = run_actions_from_regex_result(extract_steps(plan_result), user_request="휴학신청", obs_map=obs_map, max_new_tokens=256)

for sid, tplan, action in actions:
    print("------------------------------------------------")
    print(f"[STEP {sid}] {tplan}")
    print("[ACTION RESULT]\n", action)


In [ ]:
from huggingface_hub import login
login()


In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import HfApi, create_repo, upload_folder

BASE_REPO = "leeChaerin/PlanModel_v1"      # 학습 때 사용한 베이스
ADAPTER_DIR = "./adapters/epoch_009"       # 저장된 LoRA 어댑터 폴더
HF_REPO_ID = "leeChaerin/ActionModel_v5"  # 새로 만들 허깅페이스 repo 이름

# (1) 허브에 리포지토리 생성(이미 있으면 skip=True)
create_repo(repo_id=HF_REPO_ID, private=False, exist_ok=True)

# (2) 어댑터 폴더 업로드
upload_folder(
    repo_id=HF_REPO_ID,
    folder_path=ADAPTER_DIR,
    path_in_repo=".", # 루트에 업로드
)
print(f"[OK] LoRA adapter uploaded -> https://huggingface.co/{HF_REPO_ID}")
